In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv("movies.csv")
tags = pd.read_csv("tags.csv")

movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
tags.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [3]:
Ntags = tags.groupby(["movieId"])["tag"].apply(lambda x: ''.join(str(x)))

In [4]:
Ntags.to_frame()

,tag
movieId,
1,2306 Owned\n3749 ...
2,10005 Robin Williams\n10006 ...
3,667 funny\...
4,39595 based on novel or book\n39596 ...
5,39600 aging\n39601 ...
...,...
208813,"622336 might like\nName: 208813, dtype: object"
208933,669164 black and white\n669165 deal ...
209035,1009674 computer animation\n1009675 ...


In [5]:
movies = movies.merge(Ntags, on='movieId')
movies.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2306 Owned\n3749 ...
1,2,Jumanji (1995),Adventure|Children|Fantasy,10005 Robin Williams\n10006 ...
2,3,Grumpier Old Men (1995),Comedy|Romance,667 funny\...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,39595 based on novel or book\n39596 ...
4,5,Father of the Bride Part II (1995),Comedy,39600 aging\n39601 ...


In [6]:
movies.head().style.set_properties(subset=['tag'], **{'width': '500px'})

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"2306 Owned 3749 imdb top 250 4226 Pixar 10003 Pixar 10004 time travel ... 1084685 American Animation 1084688 computer animation 1084689 pixar 1088052 Pixar 1091307 animation Name: 1, Length: 697, dtype: object"
1,2,Jumanji (1995),Adventure|Children|Fantasy,"10005 Robin Williams 10006 time travel 25228 fantasy 39585 based on children's book 39586 board game ... 1013267 time travel 1048202 For children 1072418 animals 1085507 board game 1085508 Robin Williams Name: 2, Length: 180, dtype: object"
2,3,Grumpier Old Men (1995),Comedy|Romance,"667 funny 39591 best friend 39592 duringcreditsstinger 39593 fishing 39594 old man 251625 sequel fever 282638 moldy 282639 old 304982 sequel 400113 NO_FA_GANES 408920 Jack Lemmon 408921 Walter Matthau 409617 grun running 431320 comedinha de velhinhos engraÃƒÂ§ada 536456 old people that is actually funny 545427 Howard Deutch 600338 CLV 726034 sequel 789263 comedinha de velhinhos engraÃ§ada 854555 moldy 854556 old 908943 Ann Margaret 908944 Burgess Meredith 908945 Daryl Hannah 908946 fishing 908947 good soundtrack 908948 Jack Lemmon 908949 Sophia Loren 908950 Walter Matthau Name: 3, dtype: object"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,"39595 based on novel or book 39596 chick flick 39597 divorce 39598 interracial relationship 39599 single mother 600339 CLV 878412 chick flick 878413 revenge 955705 characters 1072419 characters 1072420 chick flick Name: 4, dtype: object"
4,5,Father of the Bride Part II (1995),Comedy,"39600 aging 39601 baby 39602 confidence 39603 contraception 39604 daughter 39605 gynecologist 39606 midlife crisis 39607 parent child relationship 39608 pregnancy 251626 sequel fever 409618 Fantasy 422689 steve martin 576339 Comedy 576340 Touching 600340 CLV 669376 steve martin 698661 family 698662 pregnancy 698663 wedding 878414 Diane Keaton 878415 family 878416 sequel 878417 Steve Martin 878418 wedding Name: 5, dtype: object"


In [7]:
import re

def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title

def clean_genre(genre):
    genre = re.sub("[^a-zA-Z0-9 ]", " ", genre)
    return genre

def clean_tag(tag):
    tag = re.sub(r'[^a-zA-Z]', ' ', tag)
    tag = re.sub("Name: ", " ", tag)
    tag = re.sub(", Length: ", " ", tag)
    tag = re.sub(", dtype: object", " ", tag)
    
    return tag

movies["clean_title"] = movies["title"].apply(clean_title)
movies["clean_genres"] = movies["genres"].apply(clean_genre)
movies["clean_tag"] = movies["tag"].apply(clean_tag)
movies["clean_tag"] = movies["clean_tag"].replace(r'\n',' ', regex=True) 
movies = movies[movies["clean_genres"].str.contains('no genres listed') == False]
movies

,movieId,title,genres,tag,clean_title,clean_genres,clean_tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2306 Owned\n3749 ...,Toy Story 1995,Adventure Animation Children Comedy Fantasy,Owned ...
1,2,Jumanji (1995),Adventure|Children|Fantasy,10005 Robin Williams\n10006 ...,Jumanji 1995,Adventure Children Fantasy,Robin Williams ...
2,3,Grumpier Old Men (1995),Comedy|Romance,667 funny\...,Grumpier Old Men 1995,Comedy Romance,funny ...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,39595 based on novel or book\n39596 ...,Waiting to Exhale 1995,Comedy Drama Romance,based on novel or book ...
4,5,Father of the Bride Part II (1995),Comedy,39600 aging\n39601 ...,Father of the Bride Part II 1995,Comedy,aging ...
...,...,...,...,...,...,...,...
45244,208800,Lady and the Tramp (2019),Comedy|Romance,"622335 might like\nName: 208800, dtype: object",Lady and the Tramp 2019,Comedy Romance,might like Name dtype object
45245,208804,Spell (2018),Comedy|Drama|Thriller,751297 Music\n751298 Patrick Stu...,Spell 2018,Comedy Drama Thriller,Music Patrick Stum...
45246,208813,Noelle (2019),Children,"622336 might like\nName: 208813, dtype: object",Noelle 2019,Children,might like Name dtype object
45247,208933,The Devil's Partner (1961),Horror,669164 black and white\n669165 deal ...,The Devils Partner 1961,Horror,black and white deal w...


In [8]:
#movies['RTags'] =  movies['clean_title'] + ' ' + movies['clean_genres'] + movies['clean_tag'] 
movies['RTags'] =  movies['clean_title'] + ' ' + movies['clean_genres']


new = movies.drop(columns=['genres','tag','clean_title','clean_genres','clean_tag'])
new.head(5).style.set_properties(subset=['RTags'], **{'width': '800px'})

,movieId,title,RTags
0,1,Toy Story (1995),Toy Story 1995 Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Jumanji 1995 Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Grumpier Old Men 1995 Comedy Romance
3,4,Waiting to Exhale (1995),Waiting to Exhale 1995 Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Father of the Bride Part II 1995 Comedy


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["RTags"])

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    
    return results

In [11]:
import ipywidgets as widgets
from IPython.display import display

movie_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [12]:
# Sample Movie
movie_id = 89745

#def find_similar_movies(movie_id):
movie = movies[movies["movieId"] == movie_id]
ratings = pd.read_csv("ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [13]:
# Finding users who have watched and liked the movie
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
similar_users

array([     3,     21,     50, ..., 162512, 162516, 162532], dtype=int64)

In [14]:
# Finding movies liked by similar users
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] >= 4)]["movieId"]
similar_user_recs

254              1
255             29
256             32
257             50
258            111
             ...  
24998519     96610
24998521     97921
24998522     98809
24998523    102125
24998524    112852
Name: movieId, Length: 1801838, dtype: int64

In [15]:
# Finding percentage of users that recommend that movie
similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
similar_user_recs

89745     1.000000
58559     0.708627
79132     0.684536
2571      0.665968
59315     0.655780
            ...   
165571    0.000095
177185    0.000095
187957    0.000095
128143    0.000095
7739      0.000095
Name: movieId, Length: 27743, dtype: float64

In [16]:
# Retaining top 90% of user recommendations
similar_user_recs = similar_user_recs[similar_user_recs > .10]
similar_user_recs

89745     1.000000
58559     0.708627
79132     0.684536
2571      0.665968
59315     0.655780
            ...   
38061     0.101504
61132     0.101219
88810     0.101028
4720      0.100743
103042    0.100457
Name: movieId, Length: 370, dtype: float64

In [17]:
# Finding all the users that liked the recommended movies
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] >= 4)]
    
#Finding percentage of users that recommend the movie
all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
rec_percentages.columns = ["similar", "all"]
rec_percentages

,similar,all
89745,1.000000,0.065662
58559,0.708627,0.201418
79132,0.684536,0.185193
2571,0.665968,0.348516
59315,0.655780,0.099681
...,...,...
38061,0.101504,0.027573
61132,0.101219,0.018438
88810,0.101028,0.023971
4720,0.100743,0.046054


In [18]:
# Generating score 
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
rec_percentages = rec_percentages.sort_values("score", ascending=False)
rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

,score,title,genres
16133,15.229575,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
23034,12.636510,Avengers: Age of Ultron (2015),Action|Adventure|Sci-Fi
19121,12.482574,Thor: The Dark World (2013),Action|Adventure|Fantasy|IMAX
18376,11.920596,Iron Man 3 (2013),Action|Sci-Fi|Thriller|IMAX
19854,11.672371,Captain America: The Winter Soldier (2014),Action|Adventure|Sci-Fi|IMAX
23047,11.564240,Captain America: Civil War (2016),Action|Sci-Fi|Thriller
15830,11.545860,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War
23037,10.994667,Ant-Man (2015),Action|Adventure|Sci-Fi
18516,10.907809,Man of Steel (2013),Action|Adventure|Fantasy|Sci-Fi|IMAX
15469,10.817268,Thor (2011),Action|Adventure|Drama|Fantasy|IMAX


In [19]:
def find_similar_movies(movie_id):
    # Finding users who have watched and liked the movie
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
    
    # Finding movies liked by similar users
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] >= 4)]["movieId"]
    
    # Finding percentage of users that recommend that movie
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    
    # Retaining top 90% of user recommendations
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    # Finding all the users that liked the recommended movies
    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] >= 4)]
    
    #Finding percentage of users that recommend the movie
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    # Generating score 
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [20]:
import ipywidgets as widgets
from IPython.display import display

movie_name_input = widgets.Text(
    value='Toy Story',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()